In [1]:
import pandas as pd
from IPython.display import clear_output

import os

!pip install eiapy

os.environ['EIA_KEY'] = '88f8dcdae0e5dd59a7a693365d0cc12d'
API_KEY = os.environ['EIA_KEY']

from eiapy import Category, Series

import time

## Setup to Problems

I want to understand the various energy generation methods and thier relationship to energy sector firms listed on NASDQ.  I will source this information via EIA API https://www.eia.gov/


This will focus on power productions and that is Category #3 of the API this is the list of how that content is organized.

In [2]:
cat = Category(3) #https://www.eia.gov/opendata/qb.php?category=3 # Electricity by Fuel Type

cat.get_info().keys()
#get a list of category id's for all power generation in us
for category in cat.get_info()['category']['childcategories']:


    print (category)
    
    
    

{'category_id': 1736519, 'name': 'All fuels'}
{'category_id': 4, 'name': 'Coal'}
{'category_id': 7, 'name': 'Petroleum liquids'}
{'category_id': 8, 'name': 'Petroleum coke'}
{'category_id': 9, 'name': 'Natural gas'}
{'category_id': 10, 'name': 'Other gases'}
{'category_id': 11, 'name': 'Nuclear'}
{'category_id': 12, 'name': 'Conventional hydroelectric'}
{'category_id': 13, 'name': 'Other renewables (total)'}
{'category_id': 14, 'name': 'Wind'}
{'category_id': 15, 'name': 'All utility-scale solar'}
{'category_id': 1718400, 'name': 'Utility-scale photovoltaic'}
{'category_id': 1718401, 'name': 'Utility-scale thermal'}
{'category_id': 17, 'name': 'Geothermal'}
{'category_id': 16, 'name': 'Wood and wood-derived fuels'}
{'category_id': 18, 'name': 'Other biomass'}
{'category_id': 19, 'name': 'Hydro-electric pumped storage'}
{'category_id': 20, 'name': 'Other'}
{'category_id': 1718408, 'name': 'All solar'}
{'category_id': 1718409, 'name': 'Small-scale solar photovoltaic'}


In [3]:
cat = Category(3)  #https://www.eia.gov/opendata/qb.php?category=3


#get a list of category id's for all power generation in us
cat.get_info()['category']['childcategories']

results =[]
#Target Categories
for category in cat.get_info()['category']['childcategories']:
    print (category['name'])
    
    cat = Category(category['category_id'])
    if cat == 8:
        print ('Skipping')
        continue
        
        
    for series in cat.get_info()['category']['childseries']:
        iid = series['series_id']
        #print (iid)
        
        #ignore all
        #ELEC.GEN.ALL-AK-99.M
        #if iid.find('DPV') != -1 or iid.find('ALL') != -1 :
        #    continue
        #only want Monthly Series 'M'; others 'A', 'Y'
        
        
        if iid.split('.')[-1] == 'M':
            #print (series)
            print ('\t',series['series_id'])

            ser = Series(iid)
            df = pd.DataFrame(ser.get_data(all_data=True)['series'][0]['data']
                            )
            df.columns = ['date', 'value']
            df['date'] = pd.to_datetime(df['date'], format='%Y%m')
            df['name'] = series['name']
            df['units'] = series['units']
            df['state'] = series['name'].split(':')[2].strip()
            df['type'] = series['name'].split(':')[1].strip()
            df['measure'] = series['name'].split(':')[0].strip()
            df['period'] = series['name'].split(':')[-1].strip()
            df['dt_loaded'] = pd.Timestamp.date(pd.Timestamp.now())
            
            
            
            
            results += [df]
    clear_output(wait=True)
        
clear_output(wait=True)
print ('Done!')

Done!


In [4]:
powerGenerationDF = pd.concat(results, ignore_index=True)

In [5]:
powerGenerationDF.head()

,date,value,name,units,state,type,measure,period,dt_loaded
0,2021-11-01,509.29265,Net generation : all fuels : Alaska : all sect...,thousand megawatthours,Alaska,all fuels,Net generation,monthly,2022-02-01
1,2021-10-01,472.36705,Net generation : all fuels : Alaska : all sect...,thousand megawatthours,Alaska,all fuels,Net generation,monthly,2022-02-01
2,2021-09-01,460.15390,Net generation : all fuels : Alaska : all sect...,thousand megawatthours,Alaska,all fuels,Net generation,monthly,2022-02-01
3,2021-08-01,570.55978,Net generation : all fuels : Alaska : all sect...,thousand megawatthours,Alaska,all fuels,Net generation,monthly,2022-02-01
4,2021-07-01,559.86337,Net generation : all fuels : Alaska : all sect...,thousand megawatthours,Alaska,all fuels,Net generation,monthly,2022-02-01


## Collect Some Pricing Information as well!

In [6]:
cat = Category(40)  #https://www.eia.gov/opendata/qb.php?category=3


#get a list of category id's for all power generation in us
cat.get_info()['category']['childcategories']

results =[]
#Target Categories
for category in cat.get_info()['category']['childcategories']:
    print (category['name'])
    cat = Category(category['category_id'])
    
    for series in cat.get_info()['category']['childseries']:
        iid = series['series_id']
        #print (iid)
        
        #ignore all
        #ELEC.GEN.ALL-AK-99.M
        #if iid.find('DPV') != -1 or iid.find('ALL') != -1 :
        #    continue
        #only want Monthly Series 'M'; others 'A', 'Y'
        
        
        if iid.split('.')[-1] == 'M':
            #print (series)
            print ('\t',series['series_id'])

            ser = Series(iid)
            df = pd.DataFrame(ser.get_data(all_data=True)['series'][0]['data']
                            )
            df.columns = ['date', 'value']
            df['date'] = pd.to_datetime(df['date'], format='%Y%m')
            df['name'] = series['name']
            df['units'] = series['units']
            df['consumer_type'] = series['name'].split(':')[2].strip()
            df['state'] = series['name'].split(':')[1].strip()
            df['measure'] = series['name'].split(':')[0].strip()
            df['period'] = series['name'].split(':')[-1].strip()
            df['dt_loaded'] = pd.Timestamp.date(pd.Timestamp.now())
            
            
            
            results += [df]
            
    clear_output(wait=True)
    
        
clear_output(wait=True)

print ('Done!')

Done!


In [8]:
powerPricingDF = pd.concat(results, ignore_index=True)

In [9]:
powerPricingDF.head()

,date,value,name,units,consumer_type,state,measure,period,dt_loaded
0,2021-11-01,22.60,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
1,2021-10-01,23.21,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
2,2021-09-01,23.21,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
3,2021-08-01,23.31,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
4,2021-07-01,23.42,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01


## Create BQ Table in My Personal Data Set




In [10]:
from google.cloud import bigquery

client = bigquery.Client(location="US")  


In [11]:
# Define a name for the new dataset.
dataset_id = 'us_eia'
dataset = client.create_dataset(dataset_id, exists_ok=True)  # API request


table_ref = dataset.table("electricity_generation")
try:
    dataset = client.create_dataset(dataset_id)
except:
    #Delete
    client.delete_table(table_ref, not_found_ok=True)

job = client.load_table_from_dataframe(powerGenerationDF, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/fsi-select-demo/datasets/us_eia/tables/electricity_generation


In [12]:
table_ref = dataset.table("electricity_retail_pricing")
try:
    dataset = client.create_dataset(dataset_id)
except:
    #Delete
    client.delete_table(table_ref, not_found_ok=True)

job = client.load_table_from_dataframe(powerPricingDF, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/fsi-select-demo/datasets/us_eia/tables/electricity_retail_pricing


## Create Aggregate BQ View 
Total Production by Type by Month (i.e. net out states)

Total Production by State (i.e. net out source)

In [13]:
sql = """SELECT date, type, sum(value) as total_generation 
        FROM `fsi-select-demo.us_eia.electricity_generation`
        group by date, type;
"""
table_ref = dataset.table("electricity_generation_by_type")
job_config = bigquery.QueryJobConfig(
    destination=table_ref
)

# Start the query, passing in the extra configuration.
client.delete_table(table_ref, not_found_ok=True)
    
query_job = client.query(sql, location="US", job_config=job_config)

query_job.result()  # Waits for the query to finish

    
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/fsi-select-demo/datasets/us_eia/tables/electricity_generation_by_type


In [14]:
sql = """SELECT date, state, sum(value) as total_generation 
        FROM `fsi-select-demo.us_eia.electricity_generation`
        group by date, state;
"""
table_ref = dataset.table("electricity_generation_by_state")
job_config = bigquery.QueryJobConfig(
    destination=table_ref
)

client.delete_table(table_ref, not_found_ok=True)

# Start the query, passing in the extra configuration.
query_job = client.query(sql, location="US", job_config=job_config)

query_job.result()  # Waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/fsi-select-demo/datasets/us_eia/tables/electricity_generation_by_state


In [15]:
sql = """SELECT date, consumer_type, avg(value) as retail_price_avg, min(value) as retail_price_min, max(value) as retail_price_max 
        FROM `fsi-select-demo.us_eia.electricity_retail_pricing`
        group by date, consumer_type;
"""
table_ref = dataset.table("electricity_retail_pricing_by_consumer_type")
job_config = bigquery.QueryJobConfig(
    destination=table_ref
)

client.delete_table(table_ref, not_found_ok=True)


# Start the query, passing in the extra configuration.
query_job = client.query(sql, location="US", job_config=job_config)

query_job.result()  # Waits for the query to finish
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/fsi-select-demo/datasets/us_eia/tables/electricity_retail_pricing_by_consumer_type


In [16]:
# Create a table that is a pivot.  We don't know what type of generation price point they are getting so for each date, have the types

powerPricingDF.head()

,date,value,name,units,consumer_type,state,measure,period,dt_loaded
0,2021-11-01,22.60,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
1,2021-10-01,23.21,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
2,2021-09-01,23.21,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
3,2021-08-01,23.31,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01
4,2021-07-01,23.42,Average retail price of electricity : Alaska :...,cents per kilowatthour,residential,Alaska,Average retail price of electricity,monthly,2022-02-01


In [17]:
powerPricingGroupDF = powerPricingDF[['date','state','value','consumer_type']].groupby(by=['date','state','consumer_type'], as_index=False).mean()

In [18]:
powerPricingGroupConsumerGroupDF = powerPricingGroupDF.rename_axis( axis=1)
#powerPricingGroupConsumerGroupDF

In [19]:
powerPricingGroupConsumerGroupDF = powerPricingGroupConsumerGroupDF.reset_index(0)

In [20]:
powerPricingGroupConsumerGroupDF.rename_axis( axis=1)
powerPricingGroupConsumerGroupDF = powerPricingGroupConsumerGroupDF.reset_index(0)
powerPricingGroupConsumerGroupDF = powerPricingGroupDF.pivot(index=['date', 'state'], columns='consumer_type')
powerPricingGroupConsumerGroupDF.columns = powerPricingGroupConsumerGroupDF.columns.get_level_values(-1)
powerPricingGroupConsumerGroupDF = powerPricingGroupConsumerGroupDF.reset_index()
powerPricingGroupConsumerGroupDF.head()

consumer_type,date,state,commercial,industrial,other,residential,transportation
0,2001-01-01,Alabama,6.44,3.94,6.60,6.33,NaN
1,2001-01-01,Alaska,9.70,7.05,13.36,11.22,NaN
2,2001-01-01,Arizona,6.74,4.64,4.43,6.99,0.0
3,2001-01-01,Arkansas,5.72,4.19,6.67,6.88,0.0
4,2001-01-01,California,9.28,5.75,5.42,10.89,0.0


In [21]:
table_ref = dataset.table("electricity_retail_pricing_type")
try:
    dataset = client.create_dataset(dataset_id)
except:
    #Delete
    client.delete_table(table_ref, not_found_ok=True)

job = client.load_table_from_dataframe(powerPricingGroupConsumerGroupDF, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))

Loaded dataframe to /projects/fsi-select-demo/datasets/us_eia/tables/electricity_retail_pricing_type


In [22]:
# Extra Metadata

#https://companyprofiles.carbontracker.org/NRG%20Energy  (NRG)